Test

In [10]:
%pip install python-dotenv     
%pip install langchain-community 
%pip install streamlit
%pip install langchain_ibm

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 1.9 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 2.4 MB/s eta 0:00:00a 0:00:01
  DEPRECATION: Building 'ibm-cos-sdk' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'ibm-cos-sdk'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for ibm-cos-sdk: filename

In [19]:
import os
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.
# Load environment variables from .env file
# Ensure that the .env file is in the same directory as this script or provide the path to it
APIKEY = os.getenv("APIKEY")
WATSONX_AI_URL=os.getenv("WATSONX_AI_URL")
PROJECT_ID=os.getenv("PROJECT_ID")

# Note: Be cautious with printing sensitive information like API keys in production code.
# Ensure that the .env file is not included in version control
# and is kept secure.
# print(APIKEY)  # Print the API key to verify it's loaded correctly

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.chains import RetrievalQA

# Import the WatsonxLangchain interface
from ibm_watsonx_ai import Credentials
from ibm_watsonx_ai.foundation_models import ModelInference
from ibm_watsonx_ai.foundation_models.schema import TextChatParameters

# Set up the Foundation Model on watsonx.ai

In [20]:
credentials = Credentials(
    url=WATSONX_AI_URL,
    api_key=APIKEY,
)
model_id = "mistralai/mistral-large"

### List model parameters

In [21]:
TextChatParameters.show()

+-------------------+----------------------------------------+------------------------------+
| PARAMETER         | TYPE                                   | EXAMPLE VALUE                |
+===================+========================================+==============================+
| frequency_penalty | float, NoneType                        | 0.5                          |
+-------------------+----------------------------------------+------------------------------+
| logprobs          | bool, NoneType                         | True                         |
+-------------------+----------------------------------------+------------------------------+
| top_logprobs      | int, NoneType                          | 3                            |
+-------------------+----------------------------------------+------------------------------+
| presence_penalty  | float, NoneType                        | 0.3                          |
+-------------------+---------------------------------------

In [22]:
params = TextChatParameters(
    temperature=1
)

### Initialize the model
Initialize the ModelInference class with the previously set parameters.

In [23]:
model = ModelInference(
    model_id=model_id,
    credentials=credentials,
    project_id=PROJECT_ID,
    params=params
)

In [24]:
messages = [
    {
        "role": "user", 
        "content": "What is 1 + 1"
    }
]

simple_chat_response = model.chat(messages=messages)

In [27]:
simple_chat_response

{'id': 'chatcmpl-392ae208958ba34f4ac6329ff607ba2d',
 'object': 'chat.completion',
 'model_id': 'mistralai/mistral-large',
 'model': 'mistralai/mistral-large',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': ' The sum of 1 + 1 is 2. Here it is:\n\n 1\n+1\n____\n 2'},
   'finish_reason': 'stop'}],
 'created': 1748833024,
 'model_version': '2.0.0',
 'created_at': '2025-06-02T02:57:05.877Z',
 'usage': {'completion_tokens': 29, 'prompt_tokens': 10, 'total_tokens': 39},
 'system': {'warnings': [{'message': 'This model is a Non-IBM Product governed by a third-party license that may impose use restrictions and other obligations. By using this model you agree to its terms as identified in the following URL.',
    'id': 'disclaimer_warning',
    'more_info': 'https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-models.html?context=wx'},
   {'message': "The value of 'max_completion_tokens' for this model was set to value 1024",
    'id': 'unspecified_ma

In [36]:
messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant."
    },
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "Who won the world series in 2020?"
            }
        ],
    },
    {
        "role": "assistant",
        "content": "The Los Angeles Dodgers won the World Series in 2020."
    },
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "Where was it played?"
            }
        ],
    }
]

advanced_chat_response = model.chat(messages=messages)

In [37]:
advanced_chat_response

{'id': 'chatcmpl-f902c3ec91973e3522bce683902f2765',
 'object': 'chat.completion',
 'model_id': 'mistralai/mistral-large',
 'model': 'mistralai/mistral-large',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': ' The 2020 World Series was played at Globe Life Field in Arlington, Texas. Due to the COVID-19 pandemic, the Major League Baseball (MLB) season was shortened and the World Series was played at a neutral site for the first time in modern baseball history. The Dodgers defeated the Tampa Bay Rays 4 games to 2 to win the championship.'},
   'finish_reason': 'stop'}],
 'created': 1748842411,
 'model_version': '2.0.0',
 'created_at': '2025-06-02T05:33:32.859Z',
 'usage': {'completion_tokens': 84, 'prompt_tokens': 47, 'total_tokens': 131},
 'system': {'warnings': [{'message': 'This model is a Non-IBM Product governed by a third-party license that may impose use restrictions and other obligations. By using this model you agree to its terms as identified in the 